In [1]:
import logging
import requests
import json
import pandas as pd
from datetime import date

# Start logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.debug('Start of program')

2020-07-13 09:23:01,295 - DEBUG - Start of program


In [2]:
from pyLightspeed.lsecom import api

In [3]:
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.debug('Start of program')
    
    KEY_FILE = "D:\Development\.keys\lightspeed_keys.json"

    with open(KEY_FILE) as f:
        keys = json.load(f)

    store_data = {
                'account_id': keys["account_id"],
                'save_path': 'D:\\Development\\.keys\\'
                }

    credentials = {
                'api_key': keys["api_key"],
                'api_secret': keys["api_secret"]
                }



2020-07-13 09:23:01,362 - DEBUG - Start of program


In [4]:
lse = api.Connection(store_data, credentials)

2020-07-13 09:23:01,389 - DEBUG - LS ECOM: Creating new Lightspeed Ecom to account_id : 190211


In [5]:
orders = lse.list("orders", filter="created_at_min=2020-05-15 00:00:00")

2020-07-13 09:23:01,444 - DEBUG - Starting new HTTPS connection (1): api.shoplightspeed.com:443
2020-07-13 09:23:02,021 - DEBUG - https://api.shoplightspeed.com:443 "GET /en/orders/count.json HTTP/1.1" 200 None
2020-07-13 09:23:02,076 - DEBUG - Starting new HTTPS connection (1): api.shoplightspeed.com:443
2020-07-13 09:23:03,207 - DEBUG - https://api.shoplightspeed.com:443 "GET /en/orders.json?created_at_min=2020-05-15%2000:00:00&page=1&limit=250 HTTP/1.1" 200 None
2020-07-13 09:23:03,592 - DEBUG - Starting new HTTPS connection (1): api.shoplightspeed.com:443
2020-07-13 09:23:04,076 - DEBUG - https://api.shoplightspeed.com:443 "GET /en/orders.json?created_at_min=2020-05-15%2000:00:00&page=2&limit=250 HTTP/1.1" 200 None
2020-07-13 09:23:04,278 - DEBUG - Starting new HTTPS connection (1): api.shoplightspeed.com:443
2020-07-13 09:23:04,632 - DEBUG - https://api.shoplightspeed.com:443 "GET /en/orders.json?created_at_min=2020-05-15%2000:00:00&page=3&limit=250 HTTP/1.1" 200 None


In [6]:
data =''
for order in orders:
    #if order['paymentStatus'] == 'paid' and order['status'] == 'processing_awaiting_shipment' and order['shipmentTitle'] == 'Local Delivery':
    if order['paymentStatus'] == 'paid' and order['status'] == 'processing_awaiting_shipment' and 'Local Delivery' not in order['shipmentTitle']:
        data = data + str(order['id']) + ' | '
data

'3923729 | 3911687 | 3910945 | 3910873 | 3884433 | 3863325 | 3857495 | '

In [7]:
import csv

In [8]:
outputFile = open('D:\\Data\\CloudStation\\Vintage\\ETLs\\Lightspeed Imports\\24Seven_import.csv', 'w', newline='')
outputDictWriter = csv.DictWriter(outputFile,['OrderNo','Invoice','ConsumerNo','Name','CompanyName','Add1','Add2','City','State','Zip','Phone','Weight','Email','SKU','InsuredValue','PC Type','FTT','FBTA','FBTPC','ML','SV'])
outputDictWriter.writeheader()

for order in orders:
    if order['paymentStatus'] == 'paid' and order['status'] == 'processing_awaiting_shipment' and 'Local Delivery' not in order['shipmentTitle']:
        data = {
                'OrderNo': order['id'],
                'Invoice': order['number'],
                'ConsumerNo': order['customer']['resource']['id'],
                'Name': order['addressShippingName'],
                'CompanyName':  order['addressShippingCompany'] if order['addressShippingCompany']!=False else '',
                'Add1': order['addressShippingStreet'],
                'Add2': order['addressShippingStreet'],
                'City': order['addressShippingCity'],
                'State': order['addressShippingRegionData']['code'],
                'Zip': order['addressShippingZipcode'],
                'Phone': order['phone'],
                'Weight': round(order['weight']*0.00220462,2),
                'Email': order['email'],
                'SKU':order['number'],
                'InsuredValue':100,
                'PC Type':'Wine',
                'FTT':'',
                'FBTA':'',
                'FBTPC':'',
                'ML':'',
                'SV':''
                }
        outputDictWriter.writerow(data)

outputFile.close()       
    



## In Excel


In [9]:
df = pd.DataFrame(columns=['OrderNo','Invoice','ConsumerNo','Name','CompanyName','Add1','Add2','City','State','Zip','Phone','Weight','Email','SKU','InsuredValue','PC Type','FTT','FBTA','FBTPC','ML','SV'])

In [10]:
for order in orders:
    if order['paymentStatus'] == 'paid' and order['status'] == 'processing_awaiting_shipment' and 'Local Delivery' not in order['shipmentTitle']:
        data = {
                'OrderNo': order['id'],
                'Invoice': order['number'],
                'ConsumerNo': order['customer']['resource']['id'],
                'Name': order['addressShippingName'],
                'CompanyName':  order['addressShippingCompany'] if order['addressShippingCompany']!=False else '',
                'Add1': order['addressShippingStreet'],
                'Add2': order['addressShippingStreet2'],
                'City': order['addressShippingCity'],
                'State': order['addressShippingRegionData']['code'],
                'Zip': order['addressShippingZipcode'],
                'Phone': order['phone'],
                'Weight': round(order['weight']*0.00220462,2),
                'Email': order['email'],
                'SKU':order['number'],
                'InsuredValue':100,
                'PC Type':'Wine',
                'FTT':'',
                'FBTA':'',
                'FBTPC':'',
                'ML':'',
                'SV':''
                }
        df = df.append(data, ignore_index=True)
 

In [11]:
today = date.today()
file_name = f'D:\\Data\\Vintage Team Folder\\24Seven Import Files\\VIWC_Shipping_{today.strftime("%Y_%m_%d")}.xlsx'
df.to_excel(file_name,sheet_name='Sheet1', index=False)